# Diffie Hellman

- Users A and B use the Diffie-Hellman key exchange technique with a common prime q = 71 and a
primitive root alpha = 7.
- If user A has private key XA = 5, what is A’s public key YA?
- If user B has private key XB = 12, what is B’s public key YB?
- What is the shared secret key?

In [4]:
q = 71
a = 7

Xa = 5
Xb = 12

Ya =  pow(a, Xa, q)
Yb =  pow(a, Xb, q)

print("private key a = ", Ya)
print("private key b = ", Yb)

print("Shared secret key a = ", pow(Yb, Xa, q))

private key a =  51
private key b =  4
Shared secret key a =  30


# Check Basic_DH

- We use the Python module basic_dh.py
- Create three parties a, b, c
- Let each of these parties start a Diffie-Hellman key exchange using the Diffie-Hellman group 4096, sequence 03 as defined in
basic_dh.py
- Check whether a and b , a and c, b and c succeed in generating a shared secret
- Check whether the shared secrets between ab, ac and bc are different
- Try with some other Diffie-Hellman groups, e.g.
  - 512, sequence numbers 1 to 9
  - 768, sequence numbers 1 to 6
  - 1024, sequence numbers 1 to 27
  - 1536, sequence numbers 1 to 12
  - 2048, sequence numbers 1 to 16
  - 3072, sequence numbers 1 to 4
  - 4096, sequence numbers 1 to 7
  - 6144, sequence numbers 1 to 5
  - 8192, sequence numbers 1 to 5
- Check that generating a shared secret with keys belonging to different groups does not work

In [7]:
from cryptocourse import basic_dh

a = basic_dh.DiffieHellman(4096,3)
b = basic_dh.DiffieHellman(4096,3)
c = basic_dh.DiffieHellman(4096,3)
d = basic_dh.DiffieHellman(4096,3)


print("a's private key = ", a.get_private_key())

publica = a.gen_public_key()
publicb = b.gen_public_key()
publicc = c.gen_public_key()

sharedsecretab = a.gen_shared_key(publicb)
sharedsecretba = a.gen_shared_key(publicc)


a's private key =  579931192075709260579709196082201572376018186034662487558230889797371314825593458866964645023054381413455130956021961672436479546034408293992762314740114156127907731632407026285989857545043375528198879797588039358906843602904864044718231300210096493422573380424557866196578932964502044366715752914703014663843331243228848458212322212196164327416344498698904651028120258929166069144259182963620636170915828045406788629326697753615336386240938414034780113612552064692211515552074057867073885047959819072413845182371354802774034747324636053677179358887302737918661949001401813806963405533294453826805962623301913107658094223205739682428791064279540522829204482871980790644021274502471999097251016675304196815507694277925885642564693272328596660050177664708312186744684392515157223429960720419729905812558171604256776274294529252060492190063510854240672869609102676892555860461463286987272588330174118573060383478158158774617847327498718608912563443100308926948665933624295703265809362

# Basic_DH Encrypt (xor)

- Two parties a and b perform a Diffie Hellman key exchange (take any group)
  
- A uses the portion of the shared secret with the same length as the plaintext
FEDCBAFEDCBA12345678901234567890ABCDEFABCDEF and uses XOR to symmetrically encrypt

- B uses the same portion of the shared secret and uses XOR to symmetrically decrypt the
ciphertext



In [14]:
from cryptocourse import basic_dh
from cryptocourse import basic_crypto

a = basic_dh.DiffieHellman(4096,3)
b = basic_dh.DiffieHellman(4096,3)


plain = bytes.fromhex('FEDCBAFEDCBA12345678901234567890ABCDEFABCDEF')



publica = a.gen_public_key()
publicb = b.gen_public_key()

sharedsecretab = a.gen_shared_key(publicb)
sharedsecretba = b.gen_shared_key(publica)

print(len(plain))
print(len(sharedsecretab))

part_of_stream = sharedsecretab[0:len(plain)]

cipher = basic_crypto.byte_xor(plain, part_of_stream)

part_of_streamb = sharedsecretba[0:len(cipher)]
decrypted = basic_crypto.byte_xor(cipher, part_of_streamb)

print("Plain :", plain)
print("Decrypted :",decrypted)
print ("Decrypted == Plain :", plain == decrypted)


22
32
Plain : b'\xfe\xdc\xba\xfe\xdc\xba\x124Vx\x90\x124Vx\x90\xab\xcd\xef\xab\xcd\xef'
Decrypted : b'\xfe\xdc\xba\xfe\xdc\xba\x124Vx\x90\x124Vx\x90\xab\xcd\xef\xab\xcd\xef'
Decrypted == Plain : True


# Basic_Dh encrypt (variant) -AES -> take a file as data input - HKDF

- In a variant of this exercise, use the hkdf function to generate suitable key material (256 bytes
key, 128 bytes IV, salt = b”secretpassword”, hash function: sha512) from the Diffie Hellman
shared secret and use this key for a symmetric encryption using AES-CBC. Take as plain some file
on your computer (max 2.5 Mbyte).

In [27]:
from cryptocourse import basic_dh
from cryptocourse import hkdf
from cryptocourse import aesModeOfOperation
import hashlib

a = basic_dh.DiffieHellman()
b = basic_dh.DiffieHellman()

publica = a.gen_public_key()
publicb = b.gen_public_key()

sharedsecretab = a.gen_shared_key(publicb)
sharedsecretba = b.gen_shared_key(publica)

kdf = hkdf.Hkdf(sharedsecretab, b"asecretpassword", hashlib.sha512)
key = kdf.expand(b"context", 48)

print("Length of key :",len(key))

masterkey = key[0:32]
iv = key[32:]

with open('Test.txt', 'rb') as f:
    data = f.read()

print("Length data : ",len(data))

moo = aesModeOfOperation.AESModeOfOperation()
mode, origin_len, ciph = moo.encrypt(data, moo.modeOfOperation["CBC"],masterkey,moo.aes.keySize["SIZE_256"], iv)

print("Length ciph : ",len(ciph))


decr = moo.decrypt(ciph, origin_len, mode, masterkey, moo.aes.keySize["SIZE_256"], iv)
print("decrypted == data :", decr == data)



Length of key : 48
Length data :  553
Length ciph :  560
decrypted == data : True


# Basic_Dh Adversary - Stolen private key

- Two parties a and b perform a Diffie Hellman key exchange (take any group)
- Suppose you are an adversary and you had captured a’s secret key (use the methods
get_private_key() and set_private_key() for this); you also captured b’s public key by
eavesdropping the line; calculate the shared secret yourself.

In [1]:
from cryptocourse import basic_dh
from cryptocourse import hkdf
from cryptocourse import aesModeOfOperation
import hashlib

a = basic_dh.DiffieHellman()
b = basic_dh.DiffieHellman()

publica = a.gen_public_key()
publicb = b.gen_public_key()

sharedsecretab = a.gen_shared_key(publicb)
sharedsecretba = b.gen_shared_key(publica)

stolen_private_key = a.get_private_key()
c = basic_dh.DiffieHellman()
c.set_private_key(stolen_private_key)
secret = c.gen_shared_key(publicb)

print("Secret == sharedsecret", secret == sharedsecretab)

Secret == sharedsecret True


# Basic_dh without a standard group

- Perform a standard Diffie Hellmann key exchange with length 512, but without using the standard
groups
- Make sure to create two Diffie Hellman instances sharing the initial parameters (prime and
generator)
- Generate the shared secret
- Also try it with length 2048 and see what happens


In [2]:
from cryptocourse import basic_dh
from cryptocourse import hkdf
from cryptocourse import aesModeOfOperation
import hashlib

a = basic_dh.DiffieHellman(512,0)
b = basic_dh.DiffieHellman()
b.copy(a)

publica = a.gen_public_key()
publicb = b.gen_public_key()

sharedsecretab = a.gen_shared_key(publicb)
sharedsecretba = b.gen_shared_key(publica)

print("sharedab == sharedba", sharedsecretab == sharedsecretba)

determining non standard group parameters (takes some time)
sharedab == sharedba True
